In [34]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from torcheval.metrics import R2Score
from torchmetrics.functional import r2_score
import pandas as pd
import torch
from torch.utils.data import random_split
import shutil
import os
from dataset import PGLSDataset
from models import PGLSModel, EnsemblePGLSModel
import timm

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [36]:
torch.cuda.empty_cache()

In [37]:
for folder in ["data/train_images", "data/test_images"]:
    if "0" not in os.listdir(folder):
        print("Moving images to 0 folder")
        os.makedirs(f"{folder}/0")
        for filename in os.listdir(folder):
            if filename.lower().endswith(".jpeg"):
                source_path = os.path.join(folder, filename)
                target_path = os.path.join(f"{folder}/0", filename)

                shutil.move(source_path, target_path)

In [38]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),         # Convert PIL image to tensor (H x W x C) in the range [0.0, 1.0]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

batch_size = 16     # use 4 if problem with GPU memory

In [39]:
train_images_path = 'data/train_images'
test_images_path = 'data/test_images'

train_csv_path = 'data/train.csv'
test_csv_path = 'data/test.csv'


tabular_data = pd.read_csv(train_csv_path)
targets = ["X4", "X11", "X18", "X26", "X50", "X3112"]

# Filter data
upper_values = {}
for target in targets:
    upper_values[target] = tabular_data[target+"_mean"].quantile(0.99)
    tabular_data = tabular_data[tabular_data[target+"_mean"] < upper_values[target]]
    tabular_data = tabular_data[tabular_data[target+"_mean"] > 0]

# Normalize the targets
original_means = tabular_data[[f"{target}_mean" for target in targets]].mean()
original_stds = tabular_data[[f"{target}_mean" for target in targets]].std()
tabular_data[[f"{target}_mean" for target in targets]] = (tabular_data[[f"{target}_mean" for target in targets]] - original_means) / original_stds

# Normalize the features
tabular_input_size = 0
for column in tabular_data.columns:
    if column in ["id"]+[target+"_mean" for target in targets]+[target+"_sd" for target in targets]:
        continue
    tabular_input_size += 1
    min_val = tabular_data[column].min()
    max_val = tabular_data[column].max()
    tabular_data[column] = (tabular_data[column] - min_val) / (max_val - min_val)


test_tabular_data = pd.read_csv(test_csv_path)
# Normalize the features
for column in test_tabular_data.columns:
    if column in ["id"]:
        continue
    min_val = test_tabular_data[column].min()
    max_val = test_tabular_data[column].max()
    test_tabular_data[column] = (test_tabular_data[column] - min_val) / (max_val - min_val)

train_images_dataset = ImageFolder(root=train_images_path, transform=transform)
test_images_dataset = ImageFolder(root=test_images_path, transform=transform)

train_image_csv_dataset = PGLSDataset(tabular_data=tabular_data, image_folder=train_images_dataset, transform_csv=None)
train, val = random_split(train_image_csv_dataset, [int(0.8*len(train_image_csv_dataset)), len(train_image_csv_dataset) - int(0.8*len(train_image_csv_dataset))])
test_image_csv_dataset = PGLSDataset(tabular_data=test_tabular_data, image_folder=test_images_dataset, transform_csv=None)


train_data_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_image_csv_dataset, batch_size=batch_size, shuffle=False)

In [40]:
original_stds = torch.from_numpy(original_stds.values).float()
original_means = torch.from_numpy(original_means.values).float()
def denormalize_targets(targets):
    return targets * original_stds + original_means

In [48]:
# effnet = efficientnet_b0(weights=EfficientNet_B0_Weights)
effnet = timm.create_model(
    'efficientnet_b4.ra2_in1k',
    pretrained=True,
    num_classes=0,
)

model = PGLSModel(effnet, tabular_input_size)
metric = R2Score()
criterion = torch.nn.MSELoss()

torch.cuda.empty_cache()

In [47]:
model.to(device)

optimizer = torch.optim.Adam(model.parameters(),maximize=False, lr=1e-4, weight_decay=1e-5)
max_value_metrics_batch = float("-inf")
model_best_batch_path = "./models/effnet_best_batch.pt"
max_value_metrics_epoch = float("-inf")
model_best_epoch_path = "./models/effnet_best_epoch.pt"
model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        print(loss.item())
        print(metric.compute())
        new_max_value_metrics_batch = metric.compute().item()

        if(new_max_value_metrics_batch > max_value_metrics_batch):
            max_value_metrics_batch = new_max_value_metrics_batch
            torch.save(model.state_dict(), model_best_batch_path)
    
    if(new_max_value_metrics_batch > max_value_metrics_epoch):
        max_value_metrics_epoch = new_max_value_metrics_batch
        torch.save(model.state_dict(), model_best_epoch_path)

Epoch 0
0.38923177123069763
tensor(-0.5584)
0.8054404258728027
tensor(-0.0937)
1.4053128957748413
tensor(-0.0253)
1.4774439334869385
tensor(-0.0212)
1.0113937854766846
tensor(-0.0174)
0.8494255542755127
tensor(-0.0107)
1.0444377660751343
tensor(-0.0076)
0.8111939430236816
tensor(-0.0077)
0.6304029226303101
tensor(-0.0102)
1.3158304691314697
tensor(-0.0085)
0.6532130837440491
tensor(-0.0076)
1.1371471881866455
tensor(-0.0075)
1.2428226470947266
tensor(-0.0053)
1.3151946067810059
tensor(-0.0054)
0.7168926000595093
tensor(-0.0056)
0.4788459241390228
tensor(-0.0054)
1.1843985319137573
tensor(-0.0062)
0.792550265789032
tensor(-0.0060)
1.410291075706482
tensor(-0.0058)
0.7021980285644531
tensor(-0.0053)
0.704716682434082
tensor(-0.0043)
1.379739761352539
tensor(-0.0047)
1.3159184455871582
tensor(-0.0050)
2.046245574951172
tensor(-0.0051)
1.2432420253753662
tensor(-0.0038)
0.8464863300323486
tensor(-0.0029)
0.6654527187347412
tensor(-0.0023)
0.34922027587890625
tensor(-0.0017)
0.5506973862648

KeyboardInterrupt: 

In [ ]:
accumulated_loss = 0
iterations = 0
accumulated_r2 = 0
model.to(device)
model.eval()
with torch.no_grad():
    for data in val_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        accumulated_loss += loss.item()
        iterations += 1
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        accumulated_r2 += metric.compute().item()
print("Average loss on validation set: ", accumulated_loss/iterations)
print("Average R2 on validation set: ", accumulated_r2/iterations)

Average loss on validation set:  0.8942889505368801
Average R2 on validation set:  0.08000875921474278


In [ ]:
predictions = []
model.eval()
model.to(device)
with torch.no_grad():
    for data in test_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        outputs = outputs.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        predictions.append(outputs_denorm)
predictions = [item for sublist in predictions for item in sublist]

In [ ]:
def save_predictions(predictions, test_csv_dataframe, filename):
    with open(filename, "w") as f:
        f.write("id,X4,X11,X18,X26,X50,X3112\n")
        for pred, id in zip(predictions, test_csv_dataframe["id"]):
            pred = [p.item() for p in pred]
            f.write(f"{id},{','.join([str(p) for p in pred])}\n")

In [ ]:
save_predictions(predictions, test_tabular_data, "predictions.csv")

In [49]:
# effnet = efficientnet_b0(weights=EfficientNet_B0_Weights)
effnet = timm.create_model(
    'efficientnet_b0.ra_in1k',
    pretrained=True,
    num_classes=0,
)
xception = timm.create_model('inception_resnet_v2.tf_in1k', pretrained=True, num_classes=0)
densenet = timm.create_model('densenet121.ra_in1k', pretrained=True, num_classes=0)

In [50]:

metric = R2Score()
criterion = torch.nn.MSELoss()
# criterion = r2_score
ensemble_model = EnsemblePGLSModel([effnet, xception, densenet], tabular_input_size)
torch.cuda.empty_cache()

In [51]:
ensemble_model.to(device)
max_value_metrics_batch = float("-inf")
model_best_batch_path = "./models/ensemble_best_batch.pt"
max_value_metrics_epoch = float("-inf")
model_best_epoch_path = "./models/ensemble_best_epoch.pt"
optimizer = torch.optim.Adam(ensemble_model.parameters(), lr=1e-4, weight_decay=1e-5, maximize=False)

ensemble_model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        print(loss.item())
        print(metric.compute())
        new_max_value_metrics_batch = metric.compute().item()

        if(new_max_value_metrics_batch > max_value_metrics_batch):
            max_value_metrics_batch = new_max_value_metrics_batch
            torch.save(model.state_dict(), model_best_batch_path)
    
    if(new_max_value_metrics_batch > max_value_metrics_epoch):
        max_value_metrics_epoch = new_max_value_metrics_batch
        torch.save(model.state_dict(), model_best_epoch_path)

Epoch 0


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same